# Weather Historical Data

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from IPython.display import display

C:\Users\natha\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\natha\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [86]:
# Define the file path
file_path = "C:/Users/natha/OneDrive/Documents/COMP47360-Smart-Tourist-Guide/src/data_processing/datasets/raw/weather_raw/NYCWeather.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

               time  temperature_2m (°C)  rain (mm)  snowfall (cm)  \
0  2022-01-01T00:00                -17.0        0.0           0.00   
1  2022-01-01T01:00                -15.6        0.0           0.07   
2  2022-01-01T02:00                -14.7        0.0           0.07   
3  2022-01-01T03:00                -14.2        0.0           0.07   
4  2022-01-01T04:00                -12.6        0.0           0.14   

   snow_depth (m)  weather_code (wmo code)  wind_speed_10m (km/h)  
0            0.64                        3                    8.3  
1            0.64                       71                    8.0  
2            0.64                       71                    7.6  
3            0.64                       71                    7.0  
4            0.64                       71                    7.0  


In [87]:
#Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
time                       0
temperature_2m (°C)        0
rain (mm)                  0
snowfall (cm)              0
snow_depth (m)             0
weather_code (wmo code)    0
wind_speed_10m (km/h)      0
dtype: int64


In [88]:
#Check for duplicate rows
print("Number of duplicate rows:")
print(df.duplicated().sum())

Number of duplicate rows:
0


In [89]:
#Basic descriptive statistics
print("Descriptive statistics:")
df.describe()

Descriptive statistics:


,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h)
count,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000
mean,-1.299680,0.039024,0.072657,0.548509,16.402055,5.726467
std,9.948262,0.217976,0.276860,0.572888,28.091345,2.417758
min,-38.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.300000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,-0.900000,0.000000,0.000000,0.370000,1.000000,5.500000
75%,5.700000,0.000000,0.000000,1.100000,3.000000,7.100000
max,23.200000,5.400000,4.760000,1.890000,75.000000,17.500000


In [90]:
df.head()

,time,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h)
0,2022-01-01T00:00,-17.0,0.0,0.00,0.64,3,8.3
1,2022-01-01T01:00,-15.6,0.0,0.07,0.64,71,8.0
2,2022-01-01T02:00,-14.7,0.0,0.07,0.64,71,7.6
3,2022-01-01T03:00,-14.2,0.0,0.07,0.64,71,7.0
4,2022-01-01T04:00,-12.6,0.0,0.14,0.64,71,7.0


In [91]:
df['weather_code (wmo code)'].unique()

array([ 3, 71, 73,  2,  1,  0, 75, 51, 53, 61, 55, 63], dtype=int64)

### Summary of the WMO Codes:
#### Cloud Development:

- 0: Fair
- 1: Mainly Clear
- 2: Partly Cloudy
- 3: Overcast

#### Drizzle:

- 51: Light drizzle
- 53: Moderate drizzle
- 55: Dense drizzle

#### Rain:

- 61: Light rain
- 63: Moderate rain
- 65: Heavy rain

#### Snow:

- 71: Light snow
- 73: Moderate snow
- 75: Heavy snow

In [92]:
# Dictionary mapping WMO codes to weather descriptions
wmo_codes_mapping = {
    0: 'Fair',
    1: 'Mainly Clear',
    2: 'Partly Cloudy',
    3: 'Overcast',
    51: 'Light Drizzle',
    53: 'Moderate Drizzle',
    55: 'Dense Drizzle',
    61: 'Light Rain',
    63: 'Moderate Rain',
    65: 'Heavy Rain',
    71: 'Light Snow',
    73: 'Moderate Snow',
    75: 'Heavy Snow'
}

# Map the weather codes to their descriptions
df['weather_description'] = df['weather_code (wmo code)'].map(wmo_codes_mapping)

In [93]:
#Assuming 'time' column is a string
df['datetime'] = pd.to_datetime(df['time'])
df.drop(columns=['time'], inplace=True)
# Display the DataFrame with the new 'datetime' column
df


,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h),weather_description,datetime
0,-17.0,0.0,0.00,0.64,3,8.3,Overcast,2022-01-01 00:00:00
1,-15.6,0.0,0.07,0.64,71,8.0,Light Snow,2022-01-01 01:00:00
2,-14.7,0.0,0.07,0.64,71,7.6,Light Snow,2022-01-01 02:00:00
3,-14.2,0.0,0.07,0.64,71,7.0,Light Snow,2022-01-01 03:00:00
4,-12.6,0.0,0.14,0.64,71,7.0,Light Snow,2022-01-01 04:00:00
...,...,...,...,...,...,...,...,...
17515,-20.2,0.0,0.00,0.88,0,9.4,Fair,2023-12-31 19:00:00
17516,-21.3,0.0,0.00,0.87,0,9.4,Fair,2023-12-31 20:00:00
17517,-22.2,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 21:00:00
17518,-22.9,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 22:00:00


In [94]:
# Extract year and month
df['year_month'] = df['datetime'].dt.to_period('M')

# Extract day of week (Monday=0, Sunday=6)
df['day'] = df['datetime'].dt.day

# Extract day of week (Monday=0, Sunday=6)
df['week'] = df['datetime'].dt.dayofweek

# Extract hour of the day
df['hour'] = df['datetime'].dt.hour

days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
df['week'] = df['week'].apply(lambda x: days[x])

In [95]:
df

,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h),weather_description,datetime,year_month,day,week,hour
0,-17.0,0.0,0.00,0.64,3,8.3,Overcast,2022-01-01 00:00:00,2022-01,1,Saturday,0
1,-15.6,0.0,0.07,0.64,71,8.0,Light Snow,2022-01-01 01:00:00,2022-01,1,Saturday,1
2,-14.7,0.0,0.07,0.64,71,7.6,Light Snow,2022-01-01 02:00:00,2022-01,1,Saturday,2
3,-14.2,0.0,0.07,0.64,71,7.0,Light Snow,2022-01-01 03:00:00,2022-01,1,Saturday,3
4,-12.6,0.0,0.14,0.64,71,7.0,Light Snow,2022-01-01 04:00:00,2022-01,1,Saturday,4
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,-20.2,0.0,0.00,0.88,0,9.4,Fair,2023-12-31 19:00:00,2023-12,31,Sunday,19
17516,-21.3,0.0,0.00,0.87,0,9.4,Fair,2023-12-31 20:00:00,2023-12,31,Sunday,20
17517,-22.2,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 21:00:00,2023-12,31,Sunday,21
17518,-22.9,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 22:00:00,2023-12,31,Sunday,22


In [105]:
# Check the data type of the 'year_month' column
print(df.dtypes)

temperature_2m (°C)               float64
rain (mm)                         float64
snowfall (cm)                     float64
snow_depth (m)                    float64
weather_code (wmo code)             int64
wind_speed_10m (km/h)             float64
weather_description                object
datetime                   datetime64[ns]
year_month                         object
day                                 int32
week                               object
hour                                int32
dtype: object


In [108]:
object_columns = df.select_dtypes(['object']).columns
#Convert selected columns to type 'category'
for column in object_columns:
    df[column] = df[column].astype('category')   


df["day"] = df["day"].astype('category') 
df["hour"] = df["hour"].astype('category') 

print(df.dtypes)
df

temperature_2m (°C)               float64
rain (mm)                         float64
snowfall (cm)                     float64
snow_depth (m)                    float64
weather_code (wmo code)             int64
wind_speed_10m (km/h)             float64
weather_description              category
datetime                   datetime64[ns]
year_month                       category
day                              category
week                             category
hour                             category
dtype: object


,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h),weather_description,datetime,year_month,day,week,hour
0,-17.0,0.0,0.00,0.64,3,8.3,Overcast,2022-01-01 00:00:00,2022-01,1,Saturday,0
1,-15.6,0.0,0.07,0.64,71,8.0,Light Snow,2022-01-01 01:00:00,2022-01,1,Saturday,1
2,-14.7,0.0,0.07,0.64,71,7.6,Light Snow,2022-01-01 02:00:00,2022-01,1,Saturday,2
3,-14.2,0.0,0.07,0.64,71,7.0,Light Snow,2022-01-01 03:00:00,2022-01,1,Saturday,3
4,-12.6,0.0,0.14,0.64,71,7.0,Light Snow,2022-01-01 04:00:00,2022-01,1,Saturday,4
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,-20.2,0.0,0.00,0.88,0,9.4,Fair,2023-12-31 19:00:00,2023-12,31,Sunday,19
17516,-21.3,0.0,0.00,0.87,0,9.4,Fair,2023-12-31 20:00:00,2023-12,31,Sunday,20
17517,-22.2,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 21:00:00,2023-12,31,Sunday,21
17518,-22.9,0.0,0.00,0.87,0,9.8,Fair,2023-12-31 22:00:00,2023-12,31,Sunday,22


In [109]:
# Define the list of months in your range
month_list = ["2023-12", "2023-11", "2023-10", "2023-09", "2023-08", "2023-07", "2023-06", "2023-05", "2023-04", "2023-03", "2023-02", "2023-01",
              "2022-12", "2022-11", "2022-10", "2022-09", "2022-08", "2022-07", "2022-06", "2022-05", "2022-04", "2022-03", "2022-02", "2022-01"]

# Check for rows with months not in the specified range
out_range_month = df[~df['year_month'].isin(month_list)]

# Display the rows with months not in the specified range
out_range_month

,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h),weather_description,datetime,year_month,day,week,hour


In [110]:
# Save the DataFrame to a CSV file
df.to_csv("Clean_Weather.csv", index=False)

In [112]:
# Load the CSV file into a DataFrame
df = pd.read_csv("Clean_Weather.csv")

# Display the first few rows of the DataFrame
df.head()

,temperature_2m (°C),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),wind_speed_10m (km/h),weather_description,datetime,year_month,day,week,hour
0,-17.0,0.0,0.00,0.64,3,8.3,Overcast,2022-01-01 00:00:00,2022-01,1,Saturday,0
1,-15.6,0.0,0.07,0.64,71,8.0,Light Snow,2022-01-01 01:00:00,2022-01,1,Saturday,1
2,-14.7,0.0,0.07,0.64,71,7.6,Light Snow,2022-01-01 02:00:00,2022-01,1,Saturday,2
3,-14.2,0.0,0.07,0.64,71,7.0,Light Snow,2022-01-01 03:00:00,2022-01,1,Saturday,3
4,-12.6,0.0,0.14,0.64,71,7.0,Light Snow,2022-01-01 04:00:00,2022-01,1,Saturday,4
